# Summary
Find the velocity path of movements. Velocity is calculated as the difference of consecutive points.

### imports:

In [12]:
from scipy import io
import numpy as np
import matplotlib.pyplot as plt
import statistics
import pandas as pd
import scipy.signal as signal
import scipy.interpolate as interpolate
import pickle
import glob # find files in a directory
import re # regex for string parsing

### set `folder_name` and `day`

In [13]:
folder_name = './Data/AnB1/'
day = 1
lever_data_folder = "./Data/AnB1/LeverData_ANB1_d1/"
lever_data_matfile = "./Data/AnB1/LeverData_ANB1_d1.mat"
output_folder = "./Data/AnB1/analysis5_ANB1_d1/"
analysis2_folder = "./Data/AnB1/analysis2_ANB1_d1/"
!mkdir {output_folder}

mkdir: ./Data/AnB1/analysis5_ANB1_d1/: File exists


### Define a helper function to find the moving average through convolution

In [14]:
def moving_average(fcn_values, window_length):
    return np.convolve(fcn_values, np.ones(window_length), 'valid') / window_length

### extract one movement's data, `movement_data`, from corresponding "movement_trial#.npy" file to calculate `velocity_data`
`velocity_data`: first row is times, second row is velocity data values

In [15]:
# loop through all movements from that day
for movement_file in glob.glob(analysis2_folder+'movement_trial*.npy'):
    trial_i = int(movement_file[-5])
    movement_data = np.load(movement_file)

    # the difference of consecutive points
    velocity_data = np.zeros((movement_data.shape[0], movement_data.shape[1]-1))
    velocity_data[0, :] = movement_data[0, 1:]
    velocity_data[1, :] = np.diff(movement_data[1, :])
    np.save(output_folder+'velocity_trial'+str(trial_i), velocity_data)

### Make functions scaled from 0-100% of velocity path completion to analyze velocity path variability
These scaled 0-100% velocity path functions are made by linearly interpolating the original data at rates above sampling rate to make an approximate continuous function throughout the path. Right now, the `num_samples` = 10*10000 for a minimum sampling rate of 10kHz since __the velocities all take less than 10s__.

Save all of them together in `velocities_percent_scaled` and then into "velocities_percent_scaled.npy" file.

`velocities_percent_scaled`: rows are individual velocities, columns are samples (so left to right is going forward in time)

In [18]:
# loop through all movements from that day
num_samples = 10*10000
velocities_percent_scaled = np.array([]).reshape((0, num_samples))
for velocity_file in glob.glob(output_folder+'velocity_trial*.npy'):
    trial_i = int(velocity_file[-5])
    velocity_data = np.load(velocity_file)

    percent_completion_x = np.linspace(0, 100, velocity_data[1,:].shape[0])
    velocity_data_fcn = interpolate.interp1d(percent_completion_x, velocity_data[1, :], kind='linear')
    new_x = np.linspace(0, 100, num_samples)
    new_y = velocity_data_fcn(new_x)
    velocities_percent_scaled = np.vstack([velocities_percent_scaled, new_y[:]])

np.save(output_folder+"velocities_percent_scaled", velocities_percent_scaled)
print("velocities_percent_scaled shape: ", velocities_percent_scaled.shape)

velocities_percent_scaled shape:  (10, 100000)


### calculate the movement path variance, `var_v`, across each movement in `velocities_percent_scaled`
axis=0 runs downwards across rows

In [19]:
var_v=np.var(velocities_percent_scaled, axis=0)

plot `var_p` across % movement completed

In [ ]:
x_range = np.array(range(0,num_samples)) * 100 / num_samples
plt.plot(x_range, var_p)
plt.xlabel("% movement completed")
plt.ylabel("var_p of all trials in the day")
plt.title("Variance of movement paths from first to second threshold, day "+str(day))